### 영화추천 시스템
- 영화의 줄거리 -> Tfidfvectorize -> Cosine similarity 비교

In [25]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#### 1. 데이터 탐색

In [26]:
movie = pd.read_csv('data/movies_metadata.csv', low_memory=False)
movie.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [27]:
movie.shape

(45466, 24)

In [28]:
df = movie[['title', 'overview']]
df.head()

,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...


#### 2. 데이터 전처리

In [29]:
df.isna().sum(), df.shape

(title         6
 overview    954
 dtype: int64,
 (45466, 2))

In [30]:
df.dropna(how='any', inplace=True)
df.shape

(44506, 2)

In [31]:
# title column 중복데이터 검사
df.title.nunique()

41371

In [32]:
# 중복데이터 삭제
df.drop_duplicates(subset='title', inplace=True)
df.shape

(41371, 2)

In [33]:
df.tail(3)

,title,overview
45462,Century of Birthing,An artist struggles to finish his work while a...
45464,Satan Triumphant,"In a small town live two brothers, one a minis..."
45465,Queerama,50 years after decriminalisation of homosexual...


In [34]:
# 인덱스 정리
df.set_index('title', inplace=True)
df.reset_index(inplace=True)

In [35]:
df.head()

,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...


- 모든 데이터로 하기에는 메모리 문제 발생 소지 있어
- 20000건의 데이터로 영화 추천 시스템 만들기

In [36]:
df = df.head(20000)

#### 3. 텍스트 전처리

In [37]:
# 숫자, 구둣점 제거
df['clean_doc'] = df.overview.str.replace('[^a-zA-Z]', ' ', regex=True).str.strip()
df.head()

,title,overview,clean_doc
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Led by Woody Andy s toys live happily in his ...
1,Jumanji,When siblings Judy and Peter discover an encha...,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Cheated on mistreated and stepped on the wom...
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Just when George Banks has recovered from his ...


#### 4. Feature 변환

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
overview_tv = tvect.fit_transform(df.overview)
overview_tv.shape

(20000, 47999)

In [39]:
clean_doc_tv = tvect.fit_transform(df.clean_doc)
clean_doc_tv.shape

(20000, 47120)

In [40]:
# 검색을 쉽게하기 위해서 영화 타이틀과 인덱스를 가진 테이블
indices = pd.Series(df.index, index=df.title)
indices.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

#### 5. 코사인 유사도

In [41]:
from sklearn.metrics.pairwise import linear_kernel
%time cosine_sim_overview = linear_kernel(overview_tv, overview_tv)

CPU times: total: 3.89 s
Wall time: 4.39 s


In [42]:
%time cosine_sim_clean_doc = linear_kernel(clean_doc_tv, clean_doc_tv)

CPU times: total: 6.28 s
Wall time: 9.47 s


- 영화 Jumanji와 비슷한 줄거리를 가진 영화

In [43]:
index = indices['Jumanji']
index

1

In [44]:
sim_scores = pd.Series(cosine_sim_overview[index])
sim_scores[:5]

0    0.016027
1    1.000000
2    0.049391
3    0.000000
4    0.000000
dtype: float64

In [45]:
# 나를 제외한 Top 10 score
sim_scores.sort_values(ascending=False).head(11).tail(10)

6057     0.179827
8566     0.178701
16430    0.174194
9230     0.171873
16089    0.159171
13083    0.158564
7891     0.158249
5948     0.155780
18716    0.153226
2447     0.150296
dtype: float64

In [48]:
movie_indices = sim_scores.sort_values(ascending=False).head(11).tail(10).index
movie_indices

Int64Index([6057, 8566, 16430, 9230, 16089, 13083, 7891, 5948, 18716, 2447], dtype='int64')

In [50]:
df.title.iloc[movie_indices]

6057                       Brainscan
8566                         Quintet
16430                 The Dark Angel
9230                       Word Wars
16089                         DeVour
13083    The Mindscape of Alan Moore
7891                         Masques
5948                Poolhall Junkies
18716                 Wreck-It Ralph
2447                        eXistenZ
Name: title, dtype: object

In [51]:
def get_recommendation(title, cos_sim):
    index = indices[title]
    sim_scores = pd.Series(cosine_sim_overview[index])
    movie_indices = sim_scores.sort_values(ascending=False).head(11).tail(10).index
    return df.title.iloc[movie_indices]

In [54]:
# Overview로 가져오기
get_recommendation('Toy Story', cosine_sim_overview)

14706               Toy Story 3
2945                Toy Story 2
9984     The 40 Year Old Virgin
1056      Rebel Without a Cause
11016    For Your Consideration
1910                  Condorman
3004            Man on the Moon
483                      Malice
11209              Factory Girl
16400                 Group Sex
Name: title, dtype: object

In [56]:
# Clean Doc으로 가져오기
get_recommendation('Toy Story', cosine_sim_clean_doc)

14706               Toy Story 3
2945                Toy Story 2
9984     The 40 Year Old Virgin
1056      Rebel Without a Cause
11016    For Your Consideration
1910                  Condorman
3004            Man on the Moon
483                      Malice
11209              Factory Girl
16400                 Group Sex
Name: title, dtype: object